In [ ]:
benchmark_path = '/Users/rodrigo/repos/gdock/benchmark_data/v1-1-0'
data_file = f'{benchmark_path}/benchmark_v1-1-0_noweights.dat'

In [ ]:
import glob
import os
import pandas as pd

In [ ]:
df = pd.read_table(data_file)

In [ ]:
df.head()

In [ ]:
# Single structure
top_list = [1, 5, 10, 50, 200, 400, 1000]
for top in top_list:
    d = {'high': 0, 'medium': 0, 'acceptable': 0}
    for target in df['target_name'].unique():
        subdf = df[df['target_name'] == target].sort_values(by='score')
        filtereddf = subdf.drop_duplicates(subset='score')
        topdf = filtereddf[:top]
        if (topdf['irmsd'] <= 1).sum():
            d['high'] += 1
        if (topdf['irmsd'] <= 2).sum():
            d['medium'] += 1
        if (topdf['irmsd'] <= 4).sum():
            d['acceptable'] += 1

    print('top', top, d)

print('total targets', len(df['target_name'].unique()))

In [ ]:
top_cluster_range = [1, 3, 5, 10]

for top in top_cluster_range:
    d = {'high': 0, 'medium': 0, 'acceptable': 0, 'incorrect': 0}
    for target in df['target_name'].unique():
        c_df = df[df['target_name'] == target].sort_values(['score', 'cluster_id'])
        c_df = c_df.dropna()
        c_ranking = c_df.drop_duplicates(subset='cluster_id', keep='first')
        cluster_ranking_dic = dict((i+1, j) for i, j in enumerate(c_ranking['cluster_id']) if i+1 <= top)
        
        high_l = []
        medium_l = []
        acceptable_l = []
        incorrect_l = []
        for cluster in cluster_ranking_dic:
            topdf = c_df[c_df['cluster_id'] == cluster][:5]
            if (topdf['irmsd'] <= 1).sum():
                high_l.append(1)
            if (topdf['irmsd'] <= 2).sum():
                medium_l.append(1)
            if (topdf['irmsd'] <= 4).sum():
                acceptable_l.append(1)
            elif (topdf['irmsd'] > 4).sum():
                incorrect_l.append(1)
        
        if high_l:
            d['high'] += 1
        if medium_l:
            d['medium'] += 1
        if acceptable_l:
            d['acceptable'] += 1
        elif incorrect_l:
            d['incorrect'] += 1
    print('top', top, d)
            
print('total targets', len(df['target_name'].unique()))